In [203]:
import pandas as pd
import numpy as np
from keras.optimizers import Adam
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split

from tensorflow import keras

from keras.models import Sequential
from keras.layers import Dense, Dropout

In [204]:
merged_data_path = '../data/merged_data.jsonl'
data = pd.read_json(merged_data_path, lines=True)

data.head(5)

,timestamp,user_id,track_id,session_id,track_name,popularity,duration_ms,explicit,id_artist,release_date,...,premium_user,name,genres,skipped,number_of_matching_genres,month,day_of_week,hour_of_day,genres_with_favourite_genres,common_genres
0,2023-01-03 05:09:55.000,101,2PmGtDUyJIpYBEtI1hQIVp,124,Ballrooms Of Mars,34,247707,0,3dBVyJ7JuOMt4GE9607Qin,1972-07-21,...,False,T. Rex,"[album rock, art rock, classic rock, folk rock...",False,0,1,1,5,"[album rock, art rock, classic rock, folk rock...","[protopunk, rock, glam rock, mandopop, album r..."
1,2023-02-08 21:41:02.507,103,2PmGtDUyJIpYBEtI1hQIVp,151,Ballrooms Of Mars,34,247707,0,3dBVyJ7JuOMt4GE9607Qin,1972-07-21,...,False,T. Rex,"[album rock, art rock, classic rock, folk rock...",False,0,2,2,21,"[album rock, art rock, classic rock, folk rock...","[folk, protopunk, rock, regional mexican, glam..."
2,2023-02-12 01:17:14.946,132,2PmGtDUyJIpYBEtI1hQIVp,544,Ballrooms Of Mars,34,247707,0,3dBVyJ7JuOMt4GE9607Qin,1972-07-21,...,True,T. Rex,"[album rock, art rock, classic rock, folk rock...",False,1,2,6,1,"[album rock, art rock, classic rock, folk rock...","[protopunk, rock, rock en espanol, glam rock, ..."
3,2023-01-03 03:59:59.738,132,5yxYokipsWlpDCt4Th4VVc,534,Solid Gold Easy Action,35,140067,0,3dBVyJ7JuOMt4GE9607Qin,1973-03-16,...,True,T. Rex,"[album rock, art rock, classic rock, folk rock...",False,1,1,1,3,"[album rock, art rock, classic rock, folk rock...","[protopunk, rock, rock en espanol, glam rock, ..."
4,2023-03-07 14:28:25.702,132,5yxYokipsWlpDCt4Th4VVc,547,Solid Gold Easy Action,35,140067,0,3dBVyJ7JuOMt4GE9607Qin,1973-03-16,...,True,T. Rex,"[album rock, art rock, classic rock, folk rock...",False,1,3,1,14,"[album rock, art rock, classic rock, folk rock...","[protopunk, rock, rock en espanol, glam rock, ..."


In [205]:
unique_favourite_genres = set()
for favourite_genres in data['favourite_genres']:
    unique_favourite_genres.update(favourite_genres)

unique_genres = set()
for genres in data['genres']:
    unique_genres.update(genres)

# remove genres that are not in favourite_genres
# data['genres'] = data['genres'].apply(
#     lambda x: [genre for genre in x if genre in unique_favourite_genres])

print(len(unique_genres))

data.head()

1766


,timestamp,user_id,track_id,session_id,track_name,popularity,duration_ms,explicit,id_artist,release_date,...,premium_user,name,genres,skipped,number_of_matching_genres,month,day_of_week,hour_of_day,genres_with_favourite_genres,common_genres
0,2023-01-03 05:09:55.000,101,2PmGtDUyJIpYBEtI1hQIVp,124,Ballrooms Of Mars,34,247707,0,3dBVyJ7JuOMt4GE9607Qin,1972-07-21,...,False,T. Rex,"[album rock, art rock, classic rock, folk rock...",False,0,1,1,5,"[album rock, art rock, classic rock, folk rock...","[protopunk, rock, glam rock, mandopop, album r..."
1,2023-02-08 21:41:02.507,103,2PmGtDUyJIpYBEtI1hQIVp,151,Ballrooms Of Mars,34,247707,0,3dBVyJ7JuOMt4GE9607Qin,1972-07-21,...,False,T. Rex,"[album rock, art rock, classic rock, folk rock...",False,0,2,2,21,"[album rock, art rock, classic rock, folk rock...","[folk, protopunk, rock, regional mexican, glam..."
2,2023-02-12 01:17:14.946,132,2PmGtDUyJIpYBEtI1hQIVp,544,Ballrooms Of Mars,34,247707,0,3dBVyJ7JuOMt4GE9607Qin,1972-07-21,...,True,T. Rex,"[album rock, art rock, classic rock, folk rock...",False,1,2,6,1,"[album rock, art rock, classic rock, folk rock...","[protopunk, rock, rock en espanol, glam rock, ..."
3,2023-01-03 03:59:59.738,132,5yxYokipsWlpDCt4Th4VVc,534,Solid Gold Easy Action,35,140067,0,3dBVyJ7JuOMt4GE9607Qin,1973-03-16,...,True,T. Rex,"[album rock, art rock, classic rock, folk rock...",False,1,1,1,3,"[album rock, art rock, classic rock, folk rock...","[protopunk, rock, rock en espanol, glam rock, ..."
4,2023-03-07 14:28:25.702,132,5yxYokipsWlpDCt4Th4VVc,547,Solid Gold Easy Action,35,140067,0,3dBVyJ7JuOMt4GE9607Qin,1973-03-16,...,True,T. Rex,"[album rock, art rock, classic rock, folk rock...",False,1,3,1,14,"[album rock, art rock, classic rock, folk rock...","[protopunk, rock, rock en espanol, glam rock, ..."


In [206]:
# map genres to their simpler form
def map_genre(genre):
    if 'rock' in genre:
        return 'rock'
    elif 'pop' in genre:
        return 'pop'
    elif 'folk' in genre:
        return 'folk'
    elif 'jazz' in genre:
        return 'jazz'
    elif 'metal' in genre:
        return 'metal'
    else:
        return genre

data['genres'] = data['genres'].apply(
    lambda x: [map_genre(genre) for genre in x])

data['favourite_genres'] = data['favourite_genres'].apply(
    lambda x: [map_genre(genre) for genre in x])

# delete not unique genres
data['genres'] = data['genres'].apply(lambda x: list(set(x)))
data['favourite_genres'] = data['favourite_genres'].apply(lambda x: list(set(x)))

# count unique genres
unique_genres = set()
for genres in data['genres']:
    unique_genres.update(genres)
for genres in data['favourite_genres']:
    unique_genres.update(genres)
print(len(unique_genres))

data

1151


,timestamp,user_id,track_id,session_id,track_name,popularity,duration_ms,explicit,id_artist,release_date,...,premium_user,name,genres,skipped,number_of_matching_genres,month,day_of_week,hour_of_day,genres_with_favourite_genres,common_genres
0,2023-01-03 05:09:55.000,101,2PmGtDUyJIpYBEtI1hQIVp,124,Ballrooms Of Mars,34,247707,0,3dBVyJ7JuOMt4GE9607Qin,1972-07-21,...,False,T. Rex,"[rock, protopunk]",False,0,1,1,5,"[album rock, art rock, classic rock, folk rock...","[protopunk, rock, glam rock, mandopop, album r..."
1,2023-02-08 21:41:02.507,103,2PmGtDUyJIpYBEtI1hQIVp,151,Ballrooms Of Mars,34,247707,0,3dBVyJ7JuOMt4GE9607Qin,1972-07-21,...,False,T. Rex,"[rock, protopunk]",False,0,2,2,21,"[album rock, art rock, classic rock, folk rock...","[folk, protopunk, rock, regional mexican, glam..."
2,2023-02-12 01:17:14.946,132,2PmGtDUyJIpYBEtI1hQIVp,544,Ballrooms Of Mars,34,247707,0,3dBVyJ7JuOMt4GE9607Qin,1972-07-21,...,True,T. Rex,"[rock, protopunk]",False,1,2,6,1,"[album rock, art rock, classic rock, folk rock...","[protopunk, rock, rock en espanol, glam rock, ..."
3,2023-01-03 03:59:59.738,132,5yxYokipsWlpDCt4Th4VVc,534,Solid Gold Easy Action,35,140067,0,3dBVyJ7JuOMt4GE9607Qin,1973-03-16,...,True,T. Rex,"[rock, protopunk]",False,1,1,1,3,"[album rock, art rock, classic rock, folk rock...","[protopunk, rock, rock en espanol, glam rock, ..."
4,2023-03-07 14:28:25.702,132,5yxYokipsWlpDCt4Th4VVc,547,Solid Gold Easy Action,35,140067,0,3dBVyJ7JuOMt4GE9607Qin,1973-03-16,...,True,T. Rex,"[rock, protopunk]",False,1,3,1,14,"[album rock, art rock, classic rock, folk rock...","[protopunk, rock, rock en espanol, glam rock, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9876,2023-03-12 15:00:09.124,120,62x5BmPPt987oXG84roCpq,379,Addictions,40,236800,0,3Hog7RVQ1lAv1crdfiJPLq,2009-01-01,...,True,Superbus,"[variete francaise, rock, pop]",True,0,3,6,15,"[french pop, french rock, variete francaise, b...","[french rock, lounge, country rock, french pop..."
9877,2023-03-12 15:00:09.124,120,6b84O8zIifMOPuGQDAaw01,379,有時,39,270216,0,5r0xeBSRKRJ5Dm63XzTZhE,2010-08-18,...,True,Endy Chow,[pop],False,0,3,6,15,"[c-pop, cantopop, blues rock, country rock, lo...","[lounge, cantopop, country rock, blues rock, c..."
9878,2023-03-27 22:33:16.239,120,5XkWyuHcgIQncQWMdBQ9lP,380,Join the Chant,33,364467,0,7EnAgffrVyerTWH628TJ6f,1987-01-01,...,True,Nitzer Ebb,"[electro-industrial, british industrial, ebm, ...",True,0,3,0,22,"[british industrial, ebm, electro-industrial, ...","[industrial rock, lounge, country rock, indust..."
9879,2023-03-27 22:47:11.178,120,0sCPSj1zmc6utsy0czb54e,380,Vamos hablar ingles,34,271107,0,4IYHkkHuBFwfBrqQ4XJiPA,2003-06-10,...,True,Fefita La Grande,"[tropical, merengue, merengue tipico, pop]",True,0,3,0,22,"[dominican pop, merengue, merengue tipico, tro...","[merengue tipico, lounge, country rock, domini..."


In [207]:
# balance set
skipped_data = data[data["skipped"] == 1]
not_skipped_data = data[data["skipped"] == 0]

# Split the dataset into subsets based on class labels
print("skipped_data", skipped_data.shape)
print("not_skipped_data", not_skipped_data.shape)

# Calculate the minimum number of samples among all classes
min_samples = min(len(skipped_data), len(not_skipped_data))

# Randomly select samples from each class subset to match the minimum number of samples
skipped_data_balanced = skipped_data.sample(n=min_samples, random_state=42)
not_skipped_data_balanced = not_skipped_data.sample(n=min_samples, random_state=42)

# Merge the balanced subsets to create the final balanced dataset
balanced_data = pd.concat([skipped_data_balanced, not_skipped_data_balanced], axis=0)

# Shuffle the balanced dataset
balanced_data = balanced_data.sample(frac=1, random_state=42).reset_index(drop=True)
balanced_data.head()

skipped_data (3621, 34)
not_skipped_data (6260, 34)


,timestamp,user_id,track_id,session_id,track_name,popularity,duration_ms,explicit,id_artist,release_date,...,premium_user,name,genres,skipped,number_of_matching_genres,month,day_of_week,hour_of_day,genres_with_favourite_genres,common_genres
0,2023-03-23 18:05:18.096,147,0kVB6PeqBbN2HhwJdWumeZ,732,The Wall Street Shuffle,40,236373,0,6i6WlGzQtXtz7GcC5H5st5,1974,...,True,10cc,"[mellow gold, rock, pop]",True,1,3,3,18,"[album rock, art rock, classic rock, classic u...","[rock, classic uk pop, glam rock, hoerspiel, a..."
1,2023-01-22 05:30:03.002,141,4pJKzul9oXW1lBQmOpf96m,646,Deep Dark Dungeon,38,129133,0,06nsZ3qSOYZ2hPVIMcr1IN,1981-02,...,False,J.J. Cale,"[blues, mellow gold, rock]",False,1,1,6,5,"[album rock, blues, blues rock, classic rock, ...","[rock, album rock, country rock, folk rock, cl..."
2,2023-02-23 05:54:20.102,125,7t6zMZSO6DfCeE4IcinxUJ,443,Chala Chala Navbala,0,193320,0,61JrslREXq98hurYL2hYoc,1943-12-31,...,False,Lata Mangeshkar,"[filmi, sufi, classic bollywood, pop]",False,0,2,3,5,"[classic bollywood, desi pop, filmi, sufi, cla...","[regional mexican, classic bollywood, sufi, cl..."
3,2023-02-26 04:48:49.355,149,62UVjAjo8ra1PVSeHfnxk3,754,I Love Paris,21,353200,0,3XOVABzceOUTbR3iEz0ImO,1959-01-01,...,False,Cal Tjader,"[jazz, bossa nova, lounge]",True,0,2,6,4,"[bossa nova, cool jazz, jazz, jazz funk, jazz ...","[new wave pop, latin jazz, jazz, lounge, bossa..."
4,2023-01-31 09:21:51.897,134,7DDfGy1cstvJGETyYYnfig,559,The One to Sing the Blues,30,187213,0,1DFr97A9HnbV3SKTJFu62M,1991-02-26,...,True,Motörhead,"[metal, rock]",True,0,1,1,9,"[album rock, alternative metal, hard rock, met...","[rock, album rock, brill building pop, metal, ..."


In [208]:
# Combine genres and favourite_genres
mlb = MultiLabelBinarizer()

encoded_favourite_genres = mlb.fit_transform(balanced_data['favourite_genres'])
encoded_genres = mlb.fit_transform(balanced_data['genres'])
# balanced_data.head()

In [209]:
X = np.concatenate([encoded_genres, encoded_favourite_genres], axis=1)

# Extract the labels
y = balanced_data['skipped'].astype(int).values

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True)

# count skipped and not skipped songs in X_train
print("Y_train not skipped", np.count_nonzero(y_train == 0))
print("Y_train skipped", np.count_nonzero(y_train == 1))
print("Y_train skipped %", np.count_nonzero(y_train == 1) / len(y_train))


Y_train not skipped 2874
Y_train skipped 2919
Y_train skipped % 0.5038839979285344


In [210]:
def build_model(hp):
  model = Sequential()
  model.add(Dense(10 * 2, activation='relu', input_shape=(X_train.shape[1],)))
  model.add(Dropout(0.5))
  model.add(Dense(10, activation='relu'))
  model.add(Dropout(0.5))
  model.add(Dense(1, activation='sigmoid'))

  model.compile(
      optimizer=Adam(),
      loss='binary_crossentropy', metrics=['accuracy'])
  return model


In [211]:
epochs = 100
batch_size = 128

model = build_model(None)

callbacks = [
  keras.callbacks.ModelCheckpoint(
      filepath='../models/model2_{epoch}',
      save_freq='epoch')
]

model.fit(
    X_train,
    y_train,
    validation_data=(X_test, y_test),
    epochs=epochs,
    batch_size=batch_size,
    callbacks=callbacks,
    verbose=2
)


Epoch 1/100
INFO:tensorflow:Assets written to: ../models/model2_1/assets
46/46 - 2s - loss: 0.6917 - accuracy: 0.5232 - val_loss: 0.6879 - val_accuracy: 0.5756 - 2s/epoch - 49ms/step
Epoch 2/100
INFO:tensorflow:Assets written to: ../models/model2_2/assets
46/46 - 1s - loss: 0.6860 - accuracy: 0.5438 - val_loss: 0.6807 - val_accuracy: 0.5970 - 765ms/epoch - 17ms/step
Epoch 3/100
INFO:tensorflow:Assets written to: ../models/model2_3/assets
46/46 - 1s - loss: 0.6772 - accuracy: 0.5779 - val_loss: 0.6701 - val_accuracy: 0.6259 - 662ms/epoch - 14ms/step
Epoch 4/100
INFO:tensorflow:Assets written to: ../models/model2_4/assets
46/46 - 1s - loss: 0.6632 - accuracy: 0.6007 - val_loss: 0.6582 - val_accuracy: 0.6239 - 638ms/epoch - 14ms/step
Epoch 5/100
INFO:tensorflow:Assets written to: ../models/model2_5/assets
46/46 - 1s - loss: 0.6553 - accuracy: 0.6111 - val_loss: 0.6498 - val_accuracy: 0.6404 - 615ms/epoch - 13ms/step
Epoch 6/100
INFO:tensorflow:Assets written to: ../models/model2_6/assets


In [212]:
# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, y_test, verbose=2)
print("Test set accuracy:", accuracy)

print("TEST")
y_pred = model.predict(X_test)
y_pred_classes = (y_pred > 0.5).astype(int)
print("Accuracy:", accuracy_score(y_test, y_pred_classes))
print("Confusion matrix:\n", confusion_matrix(y_test, y_pred_classes))
print("Classification report:\n", classification_report(y_test, y_pred_classes))

print("TRAIN")
y_pred = model.predict(X_train)
y_pred_classes = (y_pred > 0.5).astype(int)
print("Accuracy:", accuracy_score(y_train, y_pred_classes))
print("Confusion matrix:\n", confusion_matrix(y_train, y_pred_classes))
print("Classification report:\n", classification_report(y_train, y_pred_classes))

46/46 - 0s - loss: 0.9686 - accuracy: 0.6370 - 491ms/epoch - 11ms/step
Test set accuracy: 0.636991024017334
TEST
46/46 [==============================] - 0s 3ms/step
Accuracy: 0.6369910282953761
Confusion matrix:
 [[458 289]
 [237 465]]
Classification report:
               precision    recall  f1-score   support

           0       0.66      0.61      0.64       747
           1       0.62      0.66      0.64       702

    accuracy                           0.64      1449
   macro avg       0.64      0.64      0.64      1449
weighted avg       0.64      0.64      0.64      1449

TRAIN
182/182 [==============================] - 0s 2ms/step
Accuracy: 0.8127049887795615
Confusion matrix:
 [[2168  706]
 [ 379 2540]]
Classification report:
               precision    recall  f1-score   support

           0       0.85      0.75      0.80      2874
           1       0.78      0.87      0.82      2919

    accuracy                           0.81      5793
   macro avg       0.82      0.81 